In [1]:
# Parser Import
from data_source.parseData.googleFitDataParsing import *

In [2]:
folder_path = 'C:/Users/DELL/Desktop/ft_googlefit_codebase/local_files/takeout-20241003T064745Z-001/Takeout/Fit/All Data'
processor = ParseData(folder_path)
googleFit_df = processor.process()

In [3]:
user_name = "5cbdc5c5-2793-452b-a95d-0dbd3a8719ce"

start_date = '2024-09-28'
days_offset = 30
offset_sign = '-'
date_list = ['2024-09-13', '2024-09-09', '2024-09-07']
end_date = '2024-10-01'

# Add the new column to both DataFrames
googleFit_df['userName'] = user_name

# Reorder the columns to position 'userName' as the first column
records_columns = ['userName'] + [col for col in googleFit_df.columns if col != 'userName']

googleFit_df = googleFit_df[records_columns]

In [4]:
import pandas as pd
import os

# Correct the path by using the full absolute path
activity_logs = "C:/Users/DELL/Desktop/ft_googlefit_codebase/local_files/takeout-20241003T064745Z-001/Takeout/Fit/Daily activity metrics"

# Define the specific file to be read
file_name = "Daily activity metrics.csv"

# Construct the full file path
file_path = os.path.join(activity_logs, file_name)

# Read the specified CSV file
try:
    df_activities = pd.read_csv(file_path)

except FileNotFoundError:
    print(f"File not found: {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")


In [5]:
df_activities

,Date,Move Minutes count,Calories (kcal),Distance (m),Heart Points,Heart Minutes,Average heart rate (bpm),Max heart rate (bpm),Min heart rate (bpm),Average speed (m/s),Max speed (m/s),Min speed (m/s),Step count,Average weight (kg),Max weight (kg),Min weight (kg),Walking duration (ms),Running duration (ms),Strength training duration (ms)
0,2024-09-15,NaN,1843.233478,NaN,NaN,NaN,82.977778,103.0,67.0,NaN,NaN,NaN,1463,99.0,99.0,99.0,NaN,NaN,NaN
1,2024-09-16,67.0,2240.829028,1534.175469,65.0,65.0,81.444251,166.0,53.0,0.327074,1.230126,0.000000,5561,NaN,NaN,NaN,NaN,717000.0,3287000.0
2,2024-09-17,64.0,2242.386403,0.000000,64.0,64.0,81.193380,109.0,51.0,0.000000,0.000000,0.000000,4388,NaN,NaN,NaN,NaN,NaN,3931000.0
3,2024-09-18,60.0,2186.061224,0.000000,60.0,60.0,82.071429,125.0,51.0,0.000000,0.000000,0.000000,3776,NaN,NaN,NaN,NaN,NaN,3665000.0
4,2024-09-19,57.0,2049.328006,0.000000,57.0,57.0,83.170732,124.0,49.0,0.000000,0.000000,0.000000,2911,NaN,NaN,NaN,NaN,NaN,3465000.0
5,2024-09-20,NaN,1843.250012,NaN,NaN,NaN,80.256098,126.0,50.0,NaN,NaN,NaN,3317,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-09-21,73.0,2513.169502,33.000000,73.0,73.0,76.505415,137.0,49.0,0.015068,0.030137,0.000000,3879,NaN,NaN,NaN,1095000.0,NaN,4393000.0
7,2024-09-22,33.0,2014.416904,1339.000000,NaN,NaN,78.825368,136.0,57.0,0.699582,0.699582,0.699582,5489,NaN,NaN,NaN,1914000.0,NaN,NaN
8,2024-09-23,NaN,1843.250012,NaN,NaN,NaN,77.777003,106.0,56.0,NaN,NaN,NaN,1916,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-09-24,71.0,2206.708889,1655.149379,70.0,70.0,80.857394,138.0,54.0,0.223587,1.185022,0.000000,5162,NaN,NaN,NaN,681000.0,NaN,3563000.0


In [6]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

class TCXParser:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.data = []

    def parse_tcx_files(self):
        # Iterate over all files in the folder
        for filename in os.listdir(self.folder_path):
            if filename.endswith('.tcx'):
                file_path = os.path.join(self.folder_path, filename)
                self._parse_tcx_file(file_path)
        
        return pd.DataFrame(self.data)

    def _parse_tcx_file(self, file_path):
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # Define namespaces to simplify access
        namespaces = {
            'tcx': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
            'ns2': 'http://www.garmin.com/xmlschemas/UserProfile/v2',
            'ns3': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2',
            'ns4': 'http://www.garmin.com/xmlschemas/ProfileExtension/v1',
            'ns5': 'http://www.garmin.com/xmlschemas/ActivityGoals/v1'
        }

        for activity in root.findall('tcx:Activities/tcx:Activity', namespaces):
            activity_data = {
                'Sport': activity.get('Sport'),
                'Id': activity.find('tcx:Id', namespaces).text
            }

            for lap in activity.findall('tcx:Lap', namespaces):
                lap_data = {
                    'Lap.StartTime': lap.get('StartTime'),
                    'Lap.DistanceMeters': lap.find('tcx:DistanceMeters', namespaces).text,
                    'Lap.TotalTimeSeconds': lap.find('tcx:TotalTimeSeconds', namespaces).text,
                    'Lap.Calories': lap.find('tcx:Calories', namespaces).text,
                    'Lap.AverageHeartRateBpm': lap.find('tcx:AverageHeartRateBpm/tcx:Value', namespaces).text if lap.find('tcx:AverageHeartRateBpm/tcx:Value', namespaces) is not None else pd.NA,
                    'Lap.MaximumHeartRateBpm': lap.find('tcx:MaximumHeartRateBpm/tcx:Value', namespaces).text if lap.find('tcx:MaximumHeartRateBpm/tcx:Value', namespaces) is not None else pd.NA,
                    'Lap.Intensity': lap.find('tcx:Intensity', namespaces).text,
                    'Lap.TriggerMethod': lap.find('tcx:TriggerMethod', namespaces).text,
                }

                # Extract trackpoint data
                trackpoints = []
                for trackpoint in lap.findall('tcx:Track/tcx:Trackpoint', namespaces):
                    trackpoint_data = {
                        'Lap.Track.Trackpoint.DistanceMeters': trackpoint.find('tcx:DistanceMeters', namespaces).text,
                        'Lap.Track.Trackpoint.Time': trackpoint.find('tcx:Time', namespaces).text,
                        'HeartRateBpm': trackpoint.find('tcx:HeartRateBpm/tcx:Value', namespaces).text if trackpoint.find('tcx:HeartRateBpm/tcx:Value', namespaces) is not None else pd.NA,
                    }
                    trackpoints.append(trackpoint_data)

                # Create a record for each trackpoint and associate with lap data
                for trackpoint in trackpoints:
                    self.data.append({**activity_data, **lap_data, **trackpoint})

# Usage
folder_path = 'C:/Users/DELL/Downloads/Google Fit Data/anwesha fit data/Takeout/Fit/Activities'  # Change to your folder path
tcx_parser = TCXParser(folder_path)
activities_df = tcx_parser.parse_tcx_files()

In [7]:
# List of columns to convert
datetime_columns = ['Lap.StartTime', 'Lap.Track.Trackpoint.Time']

# Convert columns to datetime and set to IST
for column in datetime_columns:
    activities_df[column] = pd.to_datetime(activities_df[column]).dt.tz_convert('Asia/Kolkata')

activities_df

,Sport,Id,Lap.StartTime,Lap.DistanceMeters,Lap.TotalTimeSeconds,Lap.Calories,Lap.AverageHeartRateBpm,Lap.MaximumHeartRateBpm,Lap.Intensity,Lap.TriggerMethod,Lap.Track.Trackpoint.DistanceMeters,Lap.Track.Trackpoint.Time,HeartRateBpm
0,Walking,2024-09-13T13:16:51.847Z,2024-09-13 18:46:51.847000+05:30,2075.6004695563333,1773.289751035,143.521763,<NA>,<NA>,Active,Manual,0.0,2024-09-13 18:46:51.847000+05:30,NaN
1,Walking,2024-09-13T13:16:51.847Z,2024-09-13 18:46:51.847000+05:30,2075.6004695563333,1773.289751035,143.521763,<NA>,<NA>,Active,Manual,8.983634417943794,2024-09-13 18:47:21.106000+05:30,NaN
2,Walking,2024-09-13T13:16:51.847Z,2024-09-13 18:46:51.847000+05:30,2075.6004695563333,1773.289751035,143.521763,<NA>,<NA>,Active,Manual,47.094558210895656,2024-09-13 18:47:53.280000+05:30,NaN
3,Walking,2024-09-13T13:16:51.847Z,2024-09-13 18:46:51.847000+05:30,2075.6004695563333,1773.289751035,143.521763,<NA>,<NA>,Active,Manual,195.29690501388944,2024-09-13 18:49:59.354000+05:30,NaN
4,Walking,2024-09-13T13:16:51.847Z,2024-09-13 18:46:51.847000+05:30,2075.6004695563333,1773.289751035,143.521763,<NA>,<NA>,Active,Manual,208.09747735816583,2024-09-13 18:50:10.474000+05:30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,Walking,2024-10-06T15:36:34.642Z,2024-10-06 21:06:34.642000+05:30,580.843853574315,1125.225851484,62.30623027777778,<NA>,<NA>,Active,Manual,430.7062690312734,2024-10-06 21:16:45.308000+05:30,NaN
3248,Walking,2024-10-06T15:36:34.642Z,2024-10-06 21:06:34.642000+05:30,580.843853574315,1125.225851484,62.30623027777778,<NA>,<NA>,Active,Manual,475.97676208051115,2024-10-06 21:18:37.788000+05:30,NaN
3249,Walking,2024-10-06T15:36:34.642Z,2024-10-06 21:06:34.642000+05:30,580.843853574315,1125.225851484,62.30623027777778,<NA>,<NA>,Active,Manual,509.7863708134862,2024-10-06 21:20:35.788000+05:30,NaN
3250,Walking,2024-10-06T15:36:34.642Z,2024-10-06 21:06:34.642000+05:30,580.843853574315,1125.225851484,62.30623027777778,<NA>,<NA>,Active,Manual,524.1124762088145,2024-10-06 21:22:01.868000+05:30,NaN


In [8]:
import json
import os
import pandas as pd
import xml.etree.ElementTree as ET

class ParseData:
    
    def nanos_to_datetime(self, nanos):
        """Convert nanoseconds to IST datetime."""
        utc_time = pd.to_datetime(nanos, unit='ns')
        ist_time = utc_time.tz_localize('UTC').tz_convert('Asia/Kolkata')
        return ist_time.strftime('%Y-%m-%d %H:%M:%S')

    def millis_to_datetime(self, millis):
        """Convert milliseconds to IST datetime."""
        utc_time = pd.to_datetime(millis, unit='ms')
        ist_time = utc_time.tz_localize('UTC').tz_convert('Asia/Kolkata')
        return ist_time.strftime('%Y-%m-%d %H:%M:%S')

    def parse_json(self, file_path):
        """Parse a single JSON file and return a DataFrame."""
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        data_source = data.get('Data Source', '')
        data_points = data.get('Data Points', [])
        
        parsed_data = []
        for point in data_points:
            parsed_row = {}
            start_time_nanos = point.get('startTimeNanos')
            end_time_nanos = point.get('endTimeNanos')
            modified_time_millis = point.get('modifiedTimeMillis')

            parsed_row['startDate'] = self.nanos_to_datetime(start_time_nanos) if start_time_nanos else None
            parsed_row['endDate'] = self.nanos_to_datetime(end_time_nanos) if end_time_nanos else None
            parsed_row['modifiedTime'] = self.millis_to_datetime(modified_time_millis) if modified_time_millis else None

            for key, value in point.items():
                if key not in ['fitValue', 'modifiedTimeMillis']:
                    parsed_row[key] = value

            if 'fitValue' in point and point['fitValue']:
                fit_value_entry = point['fitValue'][0].get('value', None)
                if fit_value_entry:
                    for value_type, value in fit_value_entry.items():
                        parsed_row['fit_value_type'] = value_type
                        parsed_row['fit_value'] = value
                else:
                    parsed_row['fit_value_type'] = None
                    parsed_row['fit_value'] = None
            else:
                parsed_row['fit_value_type'] = None
                parsed_row['fit_value'] = None
            
            parsed_row['data_source'] = data_source
            parsed_data.append(parsed_row)

        df = pd.DataFrame(parsed_data)
        df = df.dropna(axis=1, how='all')
        return df

    def parse_tcx_file(self, file_path):
        """Parse a single TCX file and return a DataFrame."""
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        namespaces = {
            'tcx': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
            'ns2': 'http://www.garmin.com/xmlschemas/UserProfile/v2',
            'ns3': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2',
            'ns4': 'http://www.garmin.com/xmlschemas/ProfileExtension/v1',
            'ns5': 'http://www.garmin.com/xmlschemas/ActivityGoals/v1'
        }

        parsed_data = []

        for activity in root.findall('tcx:Activities/tcx:Activity', namespaces):
            activity_data = {
                'Sport': activity.get('Sport'),
                'Id': activity.find('tcx:Id', namespaces).text
            }

            for lap in activity.findall('tcx:Lap', namespaces):
                lap_data = {
                    'Lap.StartTime': lap.get('StartTime'),
                    'Lap.DistanceMeters': lap.find('tcx:DistanceMeters', namespaces).text,
                    'Lap.TotalTimeSeconds': lap.find('tcx:TotalTimeSeconds', namespaces).text,
                    'Lap.Calories': lap.find('tcx:Calories', namespaces).text,
                    'Lap.AverageHeartRateBpm': lap.find('tcx:AverageHeartRateBpm/tcx:Value', namespaces).text if lap.find('tcx:AverageHeartRateBpm/tcx:Value', namespaces) is not None else pd.NA,
                    'Lap.MaximumHeartRateBpm': lap.find('tcx:MaximumHeartRateBpm/tcx:Value', namespaces).text if lap.find('tcx:MaximumHeartRateBpm/tcx:Value', namespaces) is not None else pd.NA,
                    'Lap.Intensity': lap.find('tcx:Intensity', namespaces).text,
                    'Lap.TriggerMethod': lap.find('tcx:TriggerMethod', namespaces).text,
                }

                trackpoints = []
                for trackpoint in lap.findall('tcx:Track/tcx:Trackpoint', namespaces):
                    trackpoint_data = {
                        'Lap.Track.Trackpoint.DistanceMeters': trackpoint.find('tcx:DistanceMeters', namespaces).text,
                        'Lap.Track.Trackpoint.Time': trackpoint.find('tcx:Time', namespaces).text,
                        'HeartRateBpm': trackpoint.find('tcx:HeartRateBpm/tcx:Value', namespaces).text if trackpoint.find('tcx:HeartRateBpm/tcx:Value', namespaces) is not None else pd.NA,
                    }
                    trackpoints.append(trackpoint_data)

                for trackpoint in trackpoints:
                    parsed_data.append({**activity_data, **lap_data, **trackpoint})

        df = pd.DataFrame(parsed_data)
        df = df.dropna(axis=1, how='all')
        return df

    def parse_csv(self, file_path):
        """Parse the daily activity CSV file."""
        try:
            df = pd.read_csv(file_path)
            return df
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"An error occurred: {e}")
        return pd.DataFrame()

    def allData_json(self, folder_path):
        """Process all JSON files in the folder and return a combined DataFrame."""
        all_dfs = [self.parse_json(os.path.join(folder_path, filename)) 
                   for filename in os.listdir(folder_path) if filename.endswith('.json')]
        combined_df = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()
        return combined_df

    def activities_tcx(self, folder_path):
        """Process all TCX files in the folder and return a combined DataFrame."""
        all_dfs = [self.parse_tcx_file(os.path.join(folder_path, filename)) 
                   for filename in os.listdir(folder_path) if filename.endswith('.tcx')]
        combined_df = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()
        return combined_df

    def daily_activity_metrics(self, file_path):
        """Parse the daily activity metrics CSV and return a DataFrame."""
        return self.parse_csv(file_path)


In [9]:
json_df = ParseData().allData_json(r'C:\Users\DELL\Downloads\Google Fit Data\Hamdaan Fit Data\takeout-20241007T045109Z-002\Takeout\Fit\All Data')
tcx_df = ParseData().activities_tcx(r'C:\Users\DELL\Downloads\Google Fit Data\Hamdaan Fit Data\takeout-20241007T045109Z-002\Takeout\Fit\Activities')
csv_df = ParseData().daily_activity_metrics(r"C:\Users\DELL\Downloads\Google Fit Data\Hamdaan Fit Data\takeout-20241007T045109Z-002\Takeout\Fit\Daily activity metrics\Daily activity metrics.csv")

In [12]:
display(json_df)
display(tcx_df)
display(csv_df)

,startDate,endDate,modifiedTime,originDataSourceId,endTimeNanos,dataTypeName,startTimeNanos,rawTimestampNanos,fit_value_type,fit_value,data_source
0,2024-09-16 10:18:00,2024-09-16 10:19:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462140000000000,com.google.active_minutes,1726462080000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
1,2024-09-16 10:19:00,2024-09-16 10:20:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462200000000000,com.google.active_minutes,1726462140000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
2,2024-09-16 10:20:00,2024-09-16 10:21:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462260000000000,com.google.active_minutes,1726462200000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
3,2024-09-16 10:21:00,2024-09-16 10:22:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462320000000000,com.google.active_minutes,1726462260000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
4,2024-09-16 10:22:00,2024-09-16 10:23:00,2024-09-16 11:53:17,derived:com.google.activity.segment:com.google...,1726462380000000000,com.google.active_minutes,1726462320000000000,0,intVal,1.0,derived:com.google.active_minutes:com.google.a...
...,...,...,...,...,...,...,...,...,...,...,...
13261,2024-10-03 08:35:39,2024-10-03 09:33:56,2024-10-03 11:02:19,,1727928236000000000,com.google.step_count.delta,1727924739000000000,0,intVal,0.0,raw:com.google.step_count.delta:com.noisefit:n...
13262,2024-10-03 09:36:49,2024-10-03 09:49:43,2024-10-03 11:02:19,,1727929183000000000,com.google.step_count.delta,1727928409000000000,0,intVal,1280.0,raw:com.google.step_count.delta:com.noisefit:n...
13263,2024-10-04 08:48:20,2024-10-04 09:33:47,2024-10-04 10:33:03,,1728014627000000000,com.google.step_count.delta,1728011900000000000,0,intVal,0.0,raw:com.google.step_count.delta:com.noisefit:n...
13264,2024-10-04 09:35:47,2024-10-04 09:50:04,2024-10-04 10:33:03,,1728015604000000000,com.google.step_count.delta,1728014747000000000,0,intVal,1287.0,raw:com.google.step_count.delta:com.noisefit:n...


,Sport,Id,Lap.StartTime,Lap.DistanceMeters,Lap.TotalTimeSeconds,Lap.Calories,Lap.AverageHeartRateBpm,Lap.MaximumHeartRateBpm,Lap.Intensity,Lap.TriggerMethod,Lap.Track.Trackpoint.DistanceMeters,Lap.Track.Trackpoint.Time,HeartRateBpm
0,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T04:47:57.000Z,<NA>
1,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T04:50:00.000Z,88.0
2,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T04:55:00.000Z,105.0
3,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T05:00:00.000Z,109.0
4,Strength_training,2024-09-16T04:47:57.000Z,2024-09-16T04:47:57.000Z,0.0,3287.0,380.0,110.18181818181819,129.0,Active,Manual,0.0,2024-09-16T05:05:00.000Z,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Walking,2024-10-04T04:05:47.000Z,2024-10-04T04:05:47.000Z,874.0,857.0,101.0,114.0,134.0,Active,Manual,0.0,2024-10-04T04:05:47.000Z,<NA>
205,Walking,2024-10-04T04:05:47.000Z,2024-10-04T04:05:47.000Z,874.0,857.0,101.0,114.0,134.0,Active,Manual,874.0,2024-10-04T04:10:00.000Z,101.0
206,Walking,2024-10-04T04:05:47.000Z,2024-10-04T04:05:47.000Z,874.0,857.0,101.0,114.0,134.0,Active,Manual,874.0,2024-10-04T04:15:00.000Z,107.0
207,Walking,2024-10-04T04:05:47.000Z,2024-10-04T04:05:47.000Z,874.0,857.0,101.0,114.0,134.0,Active,Manual,874.0,2024-10-04T04:20:00.000Z,134.0


,Date,Move Minutes count,Calories (kcal),Distance (m),Heart Points,Heart Minutes,Average heart rate (bpm),Max heart rate (bpm),Min heart rate (bpm),Average speed (m/s),Max speed (m/s),Min speed (m/s),Step count,Average weight (kg),Max weight (kg),Min weight (kg),Walking duration (ms),Running duration (ms),Strength training duration (ms)
0,2024-09-15,NaN,1843.233478,NaN,NaN,NaN,82.977778,103.0,67.0,NaN,NaN,NaN,1463,99.0,99.0,99.0,NaN,NaN,NaN
1,2024-09-16,67.0,2240.829028,1534.175469,65.0,65.0,81.444251,166.0,53.0,0.327074,1.230126,0.000000,5561,NaN,NaN,NaN,NaN,717000.0,3287000.0
2,2024-09-17,64.0,2242.386403,0.000000,64.0,64.0,81.193380,109.0,51.0,0.000000,0.000000,0.000000,4388,NaN,NaN,NaN,NaN,NaN,3931000.0
3,2024-09-18,60.0,2186.061224,0.000000,60.0,60.0,82.071429,125.0,51.0,0.000000,0.000000,0.000000,3776,NaN,NaN,NaN,NaN,NaN,3665000.0
4,2024-09-19,57.0,2049.328006,0.000000,57.0,57.0,83.170732,124.0,49.0,0.000000,0.000000,0.000000,2911,NaN,NaN,NaN,NaN,NaN,3465000.0
5,2024-09-20,NaN,1843.250012,NaN,NaN,NaN,80.256098,126.0,50.0,NaN,NaN,NaN,3317,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-09-21,73.0,2513.169502,33.000000,73.0,73.0,76.505415,137.0,49.0,0.015068,0.030137,0.000000,3879,NaN,NaN,NaN,1095000.0,NaN,4393000.0
7,2024-09-22,33.0,2014.416904,1339.000000,NaN,NaN,78.825368,136.0,57.0,0.699582,0.699582,0.699582,5489,NaN,NaN,NaN,1914000.0,NaN,NaN
8,2024-09-23,NaN,1843.250012,NaN,NaN,NaN,77.777003,106.0,56.0,NaN,NaN,NaN,1916,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-09-24,71.0,2206.708889,1655.149379,70.0,70.0,80.857394,138.0,54.0,0.223587,1.185022,0.000000,5162,NaN,NaN,NaN,681000.0,NaN,3563000.0
